In [1]:
##import data_quality as dq
import openpyxl
import panel as pn
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
from zipfile import ZipFile
from importlib import reload
import matplotlib.pyplot as plt

from Framework.data_quality import DataQualityChecker
from Framework.improve_dq import Improve_DQ

In [2]:
# Download the ZIP file
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
filename = "AirQualityUCI.zip"
urllib.request.urlretrieve(url, filename)

# Extract the CSV file from the ZIP file
with ZipFile(filename, "r") as zip_file:
    csv_file = zip_file.open("AirQualityUCI.csv")

    # Read the CSV file into a DataFrame
    data = pd.read_csv(csv_file, sep=";", decimal=",")

# Create an instance of DataQualityChecker
checker = DataQualityChecker(data)

# Creatre an instance of Improve_DQ
improve = Improve_DQ(data) 

data = data.head(200)

columns_of_interest = ["CO(GT)", "PT08.S1(CO)", "NMHC(GT)", "C6H6(GT)", "PT08.S2(NMHC)", "NOx(GT)",
                       "PT08.S3(NOx)", "NO2(GT)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "RH", "AH"]


# Add expectations and calculate scores
Consistency_scores = checker.calculate_consistency_scores(columns_of_interest)
Relevancy_scores = checker.calculate_relevancy_scores(columns_of_interest, 3)



print("Check skewness data")
print(checker.check_skewness())


""" 

fig = checker.visualize_time_series(columns_of_interest)

if fig:
    plt.show()
else:
    print(f"The column '{columns_of_interest}' does not exist in the dataset.")

print("smooth_outliers")
smooth_outliers_Data = improve.smooth_outliers(columns_of_interest=columns_of_interest)
print(smooth_outliers_Data)
smooth_outliers_Data.to_excel('output.xlsx', index=False)


print("Handling duplicates")
print(improve.handle_duplicates())

print("Smoothen outlier")
print(improve.smooth_outliers())


print("Check completeness")
print(checker.check_completeness())

print("Check duplicates")
print(checker.check_duplicates())

print("Check skewtness")
print(checker.check_skewness())

print("Check missing data")
print(checker.check_missing_data())


print("Check check_stationarity column")
print(checker.check_stationarity()) """

# Define columns of interest
all_columns = data.columns.tolist()

# Create interactive dropdown for selecting columns
column_dropdown = pn.widgets.MultiChoice(
    name="Select Columns of Interest",
    options=all_columns
)

# Create placeholders for the plots and indicators
fig_consistency = pn.pane.Plotly()
fig_relevancy = pn.pane.Plotly()
circle_overall_consistency = pn.pane.Plotly()
circle_overall_relevancy = pn.pane.Plotly()

# Create a function to update plots and scores
def update_plots(event):
    selected_columns = column_dropdown.value
    selected_columns = [col for col in selected_columns if pd.api.types.is_numeric_dtype(data[col])]
    consistency_scores = checker.calculate_consistency_scores(selected_columns)
    print(type(consistency_scores))
    relevancy_scores = checker.calculate_relevancy_scores(selected_columns, 3)
    print(type(relevancy_scores))

    consistency_df = pd.DataFrame(consistency_scores, columns=["Column", "ConsistencyScore"])
    relevancy_df = pd.DataFrame(relevancy_scores, columns=["Column", "RelevancyScore"])

    # Calculate ConsistencyPercentage as a percentage from 0 to 100
    max_consistency_score = max(consistency_df["ConsistencyScore"])
    min_consistency_score = min(consistency_df["ConsistencyScore"])
    consistency_df["ConsistencyPercentage"] = ((consistency_df["ConsistencyScore"] - min_consistency_score) / (max_consistency_score - min_consistency_score) * 100).round(2)

    # Calculate RelevancyPercentage as a percentage from 0 to 100
    max_relevancy_score = max(relevancy_df["RelevancyScore"])
    min_relevancy_score = min(relevancy_df["RelevancyScore"])
    relevancy_df["RelevancyPercentage"] = ((relevancy_df["RelevancyScore"] - min_relevancy_score) / (max_relevancy_score - min_relevancy_score) * 100).round(2)


    print(consistency_df["ConsistencyPercentage"])
    print(relevancy_df["RelevancyPercentage"])
    fig_consistency.object = px.bar(consistency_df, x="Column", y="ConsistencyScore", title="Consistency Scores")
    fig_relevancy.object = px.bar(relevancy_df, x="Column", y="RelevancyScore", title="Relevancy Scores")
    
    # Calculate overall consistency and relevancy scores
    overall_consistency = consistency_df["ConsistencyPercentage"].mean()
    overall_relevancy = relevancy_df["RelevancyScore"].sum() / len(selected_columns)
    
    circle_overall_consistency.object = go.Figure(go.Indicator(
        mode="gauge+number",
        value=overall_consistency,
        title="Overall Consistency",
        domain={'x': [0, 1], 'y': [0, 1]},
        gauge={'axis': {'range': [None, 100]}}
    ))
    
    circle_overall_relevancy.object = go.Figure(go.Indicator(
        mode="gauge+number", 
        value=overall_relevancy,
        title="Overall Relevancy",
        domain={'x': [0, 1], 'y': [0, 1]},
        gauge={'axis': {'range': [None, 100]}}
    ))

# Link the dropdown widget to the update function
column_dropdown.param.watch(update_plots, "value")

# Create the layout
layout = pn.Column(
    "# Air Quality Data Quality Dashboard",
    column_dropdown,
    "## Overall Scores",
    pn.Row(
        circle_overall_consistency,
        circle_overall_relevancy
    ),
    "## Consistency Scores",
    fig_consistency,
    "## Relevancy Scores",
    fig_relevancy
)
%matplotlib
# Display the layout
layout.servable()


['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
Check missing data
Str(str)
Using matplotlib backend: <object object at 0x00000199A26BDBA0>


Column
    [0] Markdown(str)
    [1] MultiChoice(name='Select Columns o..., options=['Date', 'Time', ...])
    [2] Markdown(str)
    [3] Row
        [0] Plotly(None)
        [1] Plotly(None)
    [4] Markdown(str)
    [5] Plotly(None)
    [6] Markdown(str)
    [7] Plotly(None)